In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np
import warnings

# silencia todas as warnings sem erro de categoria
warnings.filterwarnings("ignore")


In [12]:



list_symbols = pd.Series([
     "NVDA", "GOOGL", "GOOG", "AAPL", "MSFT", "ORCL", "ADBE", "CRM", "CSCO",
    "AVGO", "AMD", "META", "UNP", "CAT", "ETN", "UPS", "ODFL", "PH", "PNC",
    "XOM", "OXY", "SHEL", "AMZN", "LOW", "HD", "CPB", "VZ", "CMCSA", "CHTR",
    "TMUS", "GD", "BILL", "PLUS", "OLED", "MRVL", "DXC", "URI", "MELI", "FANG",
    "BKR",
]).values


print(list_symbols)


['NVDA' 'GOOGL' 'GOOG' 'AAPL' 'MSFT' 'ORCL' 'ADBE' 'CRM' 'CSCO' 'AVGO'
 'AMD' 'META' 'UNP' 'CAT' 'ETN' 'UPS' 'ODFL' 'PH' 'PNC' 'XOM' 'OXY' 'SHEL'
 'AMZN' 'LOW' 'HD' 'CPB' 'VZ' 'CMCSA' 'CHTR' 'TMUS' 'GD' 'BILL' 'PLUS'
 'OLED' 'MRVL' 'DXC' 'URI' 'MELI' 'FANG' 'BKR']


In [19]:
list_melhores = []
list_relacao_mm = []
list_precos = []
for ativo in list_symbols:
    soma_fundamentos = 0
    dividend_yield = 0
    try:
        chamada_api = yf.Ticker(ativo)
        hist = chamada_api.history(period='30mo',interval = '1wk')
        hist_precos = hist['Close']
        hist_dividendos = hist['Dividends'].tz_convert(None)
        hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2023,1,1):datetime(2024,12,31)]
    except:
        continue
    

    dividend_yield = (hist_dividendos_ultimo_ano.sum()/2)/hist_precos[-1] if hist_precos[-1] > 0 else 0
    
    if dividend_yield > 0.01:
        balanco_anual = chamada_api.balance_sheet
        balanco_trimestral = chamada_api.quarterly_balance_sheet
        dre_anual = chamada_api.financials
        dfc_anual = chamada_api.cashflow
        dre_trimestral = chamada_api.quarterly_financials
        dfc_trimestral = chamada_api.quarterly_cashflow
        
        try:
            if dre_anual.keys()[0].year >= 0:
                balanco = balanco_anual
                dre = dre_anual
                dfc = dfc_anual
                variacao_receita = int(dre.loc['Total Revenue'][0] > dre.loc['Total Revenue'][1])
                variacao_ebit = int(dre.loc['EBIT'][0] > dre.loc['EBIT'][1])
                variacao_caixa = int(dfc.loc['Changes In Cash'][0] > dfc.loc['Changes In Cash'][1])
                variacao_divida = int(balanco.loc['Current Liabilities'][0] < balanco.loc['Current Liabilities'][1])
                
                soma_fundamentos = variacao_caixa + variacao_ebit + variacao_receita + variacao_divida

                if soma_fundamentos >= 2:
                    print(ativo,variacao_divida,variacao_caixa,variacao_ebit,variacao_receita,dre_anual.keys()[0])
                    list_melhores.append(ativo)
                    list_relacao_mm.append(abs(hist_precos[-1] / np.mean(hist_precos[-40:])-1))
                    list_precos.append(round(hist_precos[-1],2))
            else:
                continue
        except:
            print(ativo)

UNP 0 0 1 1 2024-12-31 00:00:00
CAT 1 0 1 0 2024-12-31 00:00:00
UPS 1 1 0 1 2024-12-31 00:00:00
PNC
SHEL 1 1 0 0 2024-12-31 00:00:00
VZ 0 1 1 1 2024-12-31 00:00:00
CMCSA 1 0 0 1 2024-12-31 00:00:00
GD 0 0 1 1 2024-12-31 00:00:00
OLED 0 1 1 1 2024-12-31 00:00:00
FANG 0 0 1 1 2024-12-31 00:00:00
BKR 0 1 1 1 2024-12-31 00:00:00


In [20]:
carteira_df = pd.DataFrame({'MM':list_relacao_mm,'Preços':list_precos}, index=list_melhores).sort_values(by = 'MM',ascending=True)

carteira_df

,MM,Preços
FANG,0.025394,148.66
OLED,0.032141,144.40
UNP,0.033670,221.97
VZ,0.041403,42.76
CMCSA,0.078905,33.23
SHEL,0.089051,72.21
BKR,0.095856,45.05
GD,0.144182,311.61
CAT,0.215773,438.02
UPS,0.217608,86.16


In [ ]:

#caso queira excluir algum ativo que se repete
#lista_drop = ['PETR3'] 

#carteira_limpa = carteira_df.drop(lista_drop)


#carteira_limpa

In [21]:
#caso rode o bloco anterior, trocar por carteira_df
list_pesos = [(1+x)/(carteira_df['MM'].sum()+len(carteira_df['MM'].values)) for x in carteira_df['MM'].values]
list_pesos.reverse()

list_pesos

[np.float64(0.11095401320994916),
 np.float64(0.11078685986048104),
 np.float64(0.10426313394322258),
 np.float64(0.09985947888355505),
 np.float64(0.09923937708685077),
 np.float64(0.09831483485753037),
 np.float64(0.09489741171278908),
 np.float64(0.09419281438200669),
 np.float64(0.09405345207097507),
 np.float64(0.09343862399264023)]

In [22]:
carteira_df['Pesos'] = list_pesos
carteira_df

,MM,Preços,Pesos
FANG,0.025394,148.66,0.110954
OLED,0.032141,144.40,0.110787
UNP,0.033670,221.97,0.104263
VZ,0.041403,42.76,0.099859
CMCSA,0.078905,33.23,0.099239
SHEL,0.089051,72.21,0.098315
BKR,0.095856,45.05,0.094897
GD,0.144182,311.61,0.094193
CAT,0.215773,438.02,0.094053
UPS,0.217608,86.16,0.093439


In [23]:
patrimonio_inicial = 10000

carteira_df['Lote'] = round((carteira_df['Pesos'] * patrimonio_inicial/carteira_df['Preços'])/100,2)*100
carteira_df

,MM,Preços,Pesos,Lote
FANG,0.025394,148.66,0.110954,7.0
OLED,0.032141,144.40,0.110787,8.0
UNP,0.033670,221.97,0.104263,5.0
VZ,0.041403,42.76,0.099859,23.0
CMCSA,0.078905,33.23,0.099239,30.0
SHEL,0.089051,72.21,0.098315,14.0
BKR,0.095856,45.05,0.094897,21.0
GD,0.144182,311.61,0.094193,3.0
CAT,0.215773,438.02,0.094053,2.0
UPS,0.217608,86.16,0.093439,11.0


In [24]:
carteira_final = carteira_df[['Lote','Preços']]
carteira_final

,Lote,Preços
FANG,7.0,148.66
OLED,8.0,144.40
UNP,5.0,221.97
VZ,23.0,42.76
CMCSA,30.0,33.23
SHEL,14.0,72.21
BKR,21.0,45.05
GD,3.0,311.61
CAT,2.0,438.02
UPS,11.0,86.16
